<a href="https://colab.research.google.com/github/sgcbayli024/Assignments/blob/main/Conor_Bayliss_Assignment_4_incomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Firstly, let us import the code from the provided file dp_algos.py

In [5]:
!pip install quantecon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 4.6 MB/s eta 0:00:00


In [1]:
import jax

In [2]:
import numpy as np
import jax.numpy as jnp

In [3]:
import matplotlib.pyplot as plt

In [6]:
import quantecon as qe
import numpy as np
from collections import namedtuple
from numba import njit, prange, int32

Value Function Iteration

In [ ]:
def value_function_iteration(v_init,
                             T,
                             get_greedy,
                             tolerance=1e-6,        # Error tolerance
                             max_iter=10_000,       # Max iteration bound
                             print_step=25,         # Print at multiples
                             verbose=True,
                             usejax=False):
    """
        Compute v_star via VFI and then compute greedy.
    """
    array_lib = jnp if usejax else np

    v = v_init
    error = tolerance + 1
    k = 1
    while error > tolerance and k <= max_iter:
        v_new = T(v)
        error = array_lib.max(array_lib.abs(v_new - v))
        if verbose and (k % print_step) == 0:
            print(f"Completed iteration {k} with error {error}.")
        v = v_new
        k += 1
    if error > tolerance:
        print(f"Warning: Iteration hit upper bound {max_iter}.")
    elif verbose:
        print(f"Terminated successfully in {k} iterations.")
    v_star = v
    σ_star = get_greedy(v_star)
    return v_star, σ_star

Optimistic Policy Iteration

In [ ]:
def optimistic_policy_iteration(v_init,
                                T_σ,
                                get_greedy,
                                tolerance=1e-6,
                                max_iter=1_000,
                                print_step=10,
                                m=60,
                                usejax=False):
    "Optimistic policy iteration routine."

    array_lib = jnp if usejax else np
    v = v_init
    error = tolerance + 1
    k = 1
    while error > tolerance and k < max_iter:
        last_v = v
        σ = get_greedy(v)
        for i in range(m):
            v = T_σ(v, σ)
        error = array_lib.max(array_lib.abs(v - last_v))
        if k % print_step == 0:
            print(f"Completed iteration {k} with error {error}.")
        k += 1
    return v, get_greedy(v)

Howard Policy Iteration

In [ ]:
def howard_policy_iteration(v_init,
                            get_value,
                            get_greedy,
                            usejax=False):
    "Howard policy iteration routine."
    array_lib = jnp if usejax else np

    σ = get_greedy(v_init)
    i, error = 0, 1.0
    while error > 0:
        v_σ = get_value(σ)
        σ_new = get_greedy(v_σ)
        error = array_lib.max(array_lib.abs(σ_new - σ))
        σ = σ_new
        i = i + 1
        print(f"Concluded loop {i} with error {error}.")
    return v_σ, σ

Next, let us import the code from the provided file model.py. Note that I have added @jax.jit to all function definitions.

In [ ]:
@jax.jit
def f(y, a, d):
    " Inventory update rule. "
    return np.maximum(y - d, 0) + a

In [ ]:
# NamedTuple to hold model parameters
Params = namedtuple(
         "Params", ("K", "c", "κ", "p"))

In [ ]:
@jax.jit
def build_R(params, y_vals, d_vals, ϕ_vals):
    " Build the R array using loops. "
    K, c, κ, p = params
    n_y = K + 1
    R = np.zeros((n_y, n_y, n_y))
    for y in y_vals:
        for yp in y_vals:
            for a in range(n_y - y):
                hits = f(y, a, d_vals) == yp
                R[y, a, yp] = np.sum(hits * ϕ_vals)
    return R

In [ ]:
@jax.jit
def build_R_vectorized(params, y_vals, d_vals, ϕ_vals):
    K, c, κ, p = params
    n_y = K + 1
    n_d = len(d_vals)
    # Create R[y, a, yp, d] and then sum out last dimension
    y  = np.reshape(y_vals, (n_y, 1, 1, 1))
    a  = np.reshape(y_vals, (1, n_y, 1, 1))
    yp = np.reshape(y_vals, (1, 1, n_y, 1))
    d  = np.reshape(d_vals, (1, 1, 1, n_d))
    ϕ  = np.reshape(ϕ_vals, (1, 1, 1, n_d))
    feasible = a <= K - y
    temp = (f(y, a, d_vals) == yp) * feasible
    R = np.sum(temp * ϕ_vals, axis=3)
    return R

In [ ]:
@jax.jit
def build_r(params, y_vals, d_vals, ϕ_vals):
    K, c, κ, p = params
    n_y = K + 1
    r = np.full((n_y, n_y), -np.inf)
    for y in y_vals:
        revenue = np.sum(np.minimum(y, d_vals) * ϕ_vals)
        for a in range(n_y - y):
            cost = c * a + κ * (a > 0)
            r[y, a] = revenue - cost
    return r

In [ ]:
@jax.jit
def build_r_vectorized(params, y_vals, d_vals, ϕ_vals):
    K, c, κ, p = params
    n_y = K + 1
    n_d = len(d_vals)
    y = np.reshape(y_vals, (n_y, 1))
    d = np.reshape(d_vals, (1, n_d))
    ϕ = np.reshape(ϕ_vals, (1, n_d))
    revenue = np.minimum(y, d) * ϕ
    exp_revenue = np.sum(revenue, axis=1)
    exp_revenue = np.reshape(exp_revenue, (n_y, 1))
    a = np.reshape(y_vals, (1, n_y))
    cost = c * a + κ * (a > 0)
    exp_profit = exp_revenue - cost
    feasible = a <= K - y
    r = np.where(feasible, exp_profit, -np.inf)
    return r

In [ ]:
# NamedTuple to hold arrays used to solve model
Arrays = namedtuple(
         "Arrays", ("r", "R", "y_vals", "z_vals", "Q"))

In [ ]:
# NamedTuple to store parameters, array sizes, and arrays
Model = namedtuple("Model", ("params", "sizes", "arrays"))

In [ ]:
@jax.jit
def create_sdd_inventory_model(ρ=0.98,        # Z persistence
                               ν=0.002,       # Z volatility
                               n_z=25,        # size of Z grid
                               b=0.97,        # Z mean
                               K=100,         # max inventory
                               d_max=100,     # max value of d
                               c=0.2,         # unit cost
                               κ=0.8,         # fixed cost
                               p=0.6):        # demand parameter

    n_y = K + 1               # size of state space
    y_vals = np.arange(n_y)   # inventory levels 0,...,K

    # Construct r and R arrays
    def ϕ(d):
        return (1 - p)**d * p
    d_vals = np.arange(d_max)
    ϕ_vals = ϕ(d_vals)

    # Build the exogenous discount process
    mc = qe.tauchen(n_z, ρ, ν)
    z_vals, Q = mc.state_values + b, mc.P
    ρL = np.max(np.abs(np.linalg.eigvals(z_vals * Q)))
    if ρL >= 1:
        raise NotImplementedError("Error: ρ(L) ≥ 1.")
    else:
        print(f"Building model with ρ(L) = {ρL}")

    # Build namedtuples and return them
    params = Params(K=K, c=c, κ=κ, p=p)
    r = build_r_vectorized(params, y_vals, d_vals, ϕ_vals)
    R = build_R_vectorized(params, y_vals, d_vals, ϕ_vals)

    arrays = Arrays(r=r, R=R, y_vals=y_vals, z_vals=z_vals, Q=Q)
    sizes = n_y, n_z
    return Model(params=params, sizes=sizes, arrays=arrays)

Next, import the code from the file plot_code.py

In [ ]:
@jax.jit
def sim_inventories(model, σ_star, ts_length, Y_init=0, seed=500):
    """
        Simulate inventory dynamics and interest rates given an
        optimal policy σ_star.
    """
    # Set up
    np.random.seed(seed)
    K, c, κ, p = model.params
    r, R, y_vals, z_vals, Q = model.arrays

    # Generate Markov chain for discount factor
    z_mc = qe.MarkovChain(Q, z_vals)
    i_z = z_mc.simulate_indices(ts_length, init=1, random_state=seed)

    # Generate corresponding inventory series
    Y = np.zeros(ts_length, dtype=int)
    Y[0] = Y_init
    for t in range(ts_length - 1):
        D = np.random.geometric(p) - 1
        a = σ_star[Y[t], i_z[t]]
        Y[t+1] = f(Y[t],  a,  D)

    # Return both series
    return Y, z_vals[i_z]

In [ ]:
@jax.jit
def plot_ts(model,
            σ_star,
            ts_length=400,
            fontsize=12,
            figname="ts.pdf",
            savefig=False):
    """
        Solve model, plot a time series of inventory and interest rates.

    """

    # Obtain inventory and discount factor series
    Y, Z = sim_inventories(model, σ_star, ts_length)
    r = (1 / Z) - 1 # calculate interest rate from discount factors

    # Plot
    fig, axes = plt.subplots(2, 1, figsize=(9, 5.5))
    ax = axes[0]
    ax.plot(Y, label="inventory", alpha=0.7)
    ax.set_xlabel("time", fontsize=fontsize)
    ax.legend(fontsize=fontsize, frameon=False)
    ax.set_ylim(0, np.max(Y)+3)
    ax = axes[1]
    ax.plot(r, label="$r_t$", alpha=0.7)
    ax.set_xlabel("$t$", fontsize=fontsize)
    ax.legend(fontsize=fontsize, frameon=False)
    plt.tight_layout()
    plt.show()
    if savefig:
        fig.savefig(figname)

In [ ]:
@jax.jit
def plot_timing(hpi_time,
                vfi_time,
                opi_times,
                m_vals,
                figname="timing.pdf",
                fontsize=12,
                savefig=False):
    """
    Plot relative timing of different algorithms.

    """
    fig, ax = plt.subplots(figsize=(9, 5.2))

    y_values = (np.full(len(m_vals), vfi_time),
                np.full(len(m_vals), hpi_time),
                opi_times)
    labels = "VFI", "HPI", "OPI"

    for y_vals, label in zip(y_values, labels):
        ax.plot(m_vals, y_vals, lw=2, label=label)

    ax.legend(fontsize=fontsize, frameon=False)
    ax.set_xlabel("$m$", fontsize=fontsize)
    ax.set_ylabel("time", fontsize=fontsize)
    plt.show()

    if savefig:
        fig.savefig(figname)

In [ ]:
f = jax.jit(f)

In [ ]:
@jax.jit
def B(y, i_z, a, v, model):
    """
    B(y, a, v) = r(y, a) + β(z) Σ_{y′, z′} v(y′, z′) R(y, a, y′) Q(z, z′)

    Jax implementation
    """
    # Set up
    K, c, κ, p = model.params
    r, R, y_vals, z_vals, Q = model.arrays
    β = z_vals[i_z]
    I, J =len(y_vals) , len(z_vals)
    N = I * J
    #Reshape and broadcast over (i, j, i', j')

    #Reshape and broadcast over (y, a, y', z, z')
    Q  = jnp.reshape(Q, (1, 1, 1, J, J))
    R  = jnp.reshape(R, (I, I, I, 1, 1))
    v  = jnp.reshape(v, (1, 1, I, 1, J))
    EV = jnp.sum(v*R*Q, axis=(2, 4))
    #Now, reshape and broadcast over  (y ,a ,z)
    β  = jnp.reshape(β, (1, 1, J))
    r  = jnp.reshape(r, (I, I, 1))
    B  = r + β * EV
    return B

In [ ]:
@jax.jit
def compute_r_σ(σ, model):
    """
    Compute the array r_σ[i, j] = r[i, j, σ[i, j]], which gives current
    rewards given policy σ.
    """

    # Unpack model
    K, c, κ, p = model.params
    r, R, y_vals, z_vals, Q = model.arrays
    z_idx=jnp.arange(z_vals)
    I, J =len(y_vals) , len(z_vals)
    N = I * J
    # Compute r_σ[i, j]
    y = jnp.reshape(y_vals, (I, 1))
    z = jnp.reshape(z_idx, (1, J))
    r_σ = r[y, σ[y,z]]

    return r_σ

In [ ]:
@jax.jit
def compute_R_σ(σ, model):
    """
    Compute the array r_σ[i, j] = r[i, j, σ[i, j]], which gives current
    rewards given policy σ.
    """

    # Unpack model
    K, c, κ, p = model.params
    r, R, y_vals, z_vals, Q = model.arrays
    z_idx=jnp.arange(z_vals)
    I, J =len(y_vals) , len(z_vals)
    N = I * J
    # Compute r_σ[i, j]
    y = jnp.reshape(y_vals, (I, 1, 1))
    z = jnp.reshape(z_idx, (1, J, 1))
    yp = jnp.reshape(y_vals, (1, 1, I))
    R_σ = R[y, σ[y,z], yp]

    return R_σ

In [ ]:
@jax.jit
def T_σ(v, σ, model):
    "The policy operator."
    K, c, κ, p = model.params
    r, R, y_vals, z_vals, Q = model.arrays
    n_y, n_z = model.sizes
    I, J =len(y_vals) , len(z_vals)
    N = I * J

    r_σ=compute_r_σ(σ, model)

    #Compute the array v_σ
    yp_idx = jnp.arange(y_vals)
    yp_idx = jnp.reshape(yp_idx, (1, 1, I))
    σ = jnp.reshape(σ, (I, J, 1))
    V = v[σ, yp_idx]

    Q  = jnp.reshape(Q, (1, 1, 1, J, J))
    R  = jnp.reshape(R, (I, I, I, 1, 1))
    v  = jnp.reshape(v, (1, 1, I, 1, J))
    Ev = jnp.sum(R*Q, axis=(2, 4))
    EV = V * Ev
    β  = jnp.reshape(β, (1, 1, J))
    return r_σ + β * EV

In [ ]:
@jax.jit
def T(v, model):
    """The Bellman operator."""
    return jnp.max(B(y, i_z, a, v, model), axis=2)

In [ ]:
@jax.jit
def get_greedy(v, constants, sizes, arrays):
    "Computes a v-greedy policy, returned as a set of indices."
    return jnp.argmax(B(v, constants, sizes, arrays), axis=2)

In [ ]:
## Custom solvers
@jax.jit
def solve_model_jax(model, algorithm="OPI", **kwargs):
    """
    General purpose solver.

    algorithm : OPI, VFI or HPI

    """

    # Set up
    n_y, n_z = model.sizes
    v_init = jnp.zeros((n_y, n_z))

    # Solve
    print(f"Solving model using {algorithm}.")
    match algorithm:
        case "OPI":
            solver = optimistic_policy_iteration
            args = (v_init,
                lambda v, σ: T_σ(v, σ, model),
                lambda v: get_greedy(v, model))
        case "HPI":
            solver = howard_policy_iteration
            args = (v_init,
                lambda σ: get_value(σ, model),
                lambda v: get_greedy(v, model))
        case "VFI":
            solver = value_function_iteration
            args = (v_init,
                lambda v: T(v, model),
                lambda v: get_greedy(v, model))
        case _:
            raise ValueError("Algorithm must be in {OPI, VFI, HPI}")

    qe.tic()
    v_star, σ_star = solver(*args, **kwargs)
    run_time = qe.toc()
    print(f"Solved model using {algorithm} in {run_time:.5f} seconds.")

    return v_star, σ_star

In [ ]:
@jax.jit
def test_timing_jax(model,
                      m_vals=range(1, 100, 20),
                      figname="numba_timing.pdf",
                      savefig=False):
    """
    Plot relative timing of different algorithms.

    """

    qe.tic()
    _, σ_pi = solve_model_jax(model, algorithm="HPI")
    hpi_time = qe.toc()

    qe.tic()
    _, σ_vfi = solve_model_jax(model, algorithm="VFI")
    vfi_time = qe.toc()

    error = jnp.max(jnp.abs(σ_vfi - σ_pi))
    if error:
        print("Warning: VFI policy deviated with max error {error}.")

    opi_times = []
    for m in m_vals:
        qe.tic()
        _, σ_opi = solve_model_jax(model, algorithm="OPI", m=m)
        opi_times.append(qe.toc())

        error = jnp.max(jnp.abs(σ_opi - σ_pi))
        if error:
            print("Warning: OPI policy deviated with max error {error}.")

    plot_timing(hpi_time,
                vfi_time,
                opi_times,
                m_vals,
                figname=figname,
                savefig=False)

    return hpi_time, vfi_time, opi_times

Now, let's solve.

In [ ]:
model = create_sdd_inventory_model()

In [ ]:
### Solve by VFI

v_star, σ_star = solve_model_jax(model, algorithm="VFI")

In [ ]:
### Solve by HPI

v_star, σ_star = solve_model_jax(model, algorithm="HPI")

In [ ]:
### Solve by OPI

v_star, σ_star = solve_model_jax(model, algorithm="OPI")

In [ ]:
plot_ts(model, σ_star, figname="jax_ts.pdf", savefig=False)

In [ ]:
### Test timing

hpi_time, vfi_time, opi_times = test_timing_jax(model)